**Webanalytics**<br/>
Prof. Dr. Jan Kirenz <br/>
Hochschule der Medien Stuttgart

In [35]:
import re
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import seaborn as sns

## 1 Import data

In [36]:
PATH = '/Users/jankirenz/Dropbox/Data/' 
df = pd.read_csv(PATH + 'Webanalytics_20100101-20181217.csv', delimiter=',', skiprows=5, decimal=',')

## 2 Tidying data

### 2.1 Data inspection

First of all, let's take a look at the variables (columns) in the data set.

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 15 columns):
Default Channel Grouping (Standard-Channelgruppierung)    1220 non-null object
Alter                                                     1220 non-null object
Land                                                      1220 non-null object
Absprungrate                                              1220 non-null object
Nutzer                                                    1220 non-null object
Seiten/Sitzung                                            1220 non-null float64
Durchschnittl. Sitzungsdauer                              1220 non-null object
Eindeutige Ereignisse                                     1220 non-null object
Hinzufügungen des Produkts zum Einkaufswagen              1220 non-null object
Purchase Completed (Abschlüsse für Zielvorhaben 1)        1220 non-null object
E-Commerce-Conversion-Rate                                1220 non-null object
Durchschn. Preis       

In [38]:
# show the first 5 rows (i.e. head of the DataFrame)
df.head(5)

,Default Channel Grouping (Standard-Channelgruppierung),Alter,Land,Absprungrate,Nutzer,Seiten/Sitzung,Durchschnittl. Sitzungsdauer,Eindeutige Ereignisse,Hinzufügungen des Produkts zum Einkaufswagen,Purchase Completed (Abschlüsse für Zielvorhaben 1),E-Commerce-Conversion-Rate,Durchschn. Preis,Durchschn. Bestellwert,Gesamtwert,Umsatz
0,Display,65+,Mexico,"100,00 %",79,1.00,00:00:00,0,0,0,"0,00 %","0,00 $","0,00 $","0,00 $","0,00 $"
1,Display,65+,Canada,"96,65 %",188,1.17,00:00:04,22,0,0,"0,00 %","0,00 $","0,00 $","0,00 $","0,00 $"
2,Social,55-64,Italy,"94,23 %",231,1.11,00:00:08,22,0,0,"0,00 %","0,00 $","0,00 $","0,00 $","0,00 $"
3,Display,35-44,Saudi Arabia,"93,50 %",101,1.11,00:00:04,0,0,0,"0,00 %","0,00 $","0,00 $","0,00 $","0,00 $"
4,Display,25-34,Argentina,"92,02 %",79,1.07,00:00:48,0,0,0,"0,00 %","0,00 $","0,00 $","0,00 $","0,00 $"


In [39]:
# show all variables in the data set
df.columns

Index(['Default Channel Grouping (Standard-Channelgruppierung)', 'Alter',
       'Land', 'Absprungrate', 'Nutzer', 'Seiten/Sitzung',
       'Durchschnittl. Sitzungsdauer', 'Eindeutige Ereignisse',
       'Hinzufügungen des Produkts zum Einkaufswagen',
       'Purchase Completed (Abschlüsse für Zielvorhaben 1)',
       'E-Commerce-Conversion-Rate', 'Durchschn. Preis',
       'Durchschn. Bestellwert', 'Gesamtwert', 'Umsatz'],
      dtype='object')

In [40]:
# Select relevant columns for data cleaning
cols = ['Absprungrate', 'E-Commerce-Conversion-Rate', 
        'Durchschn. Preis', 'Durchschn. Bestellwert', 
        'Gesamtwert', 'Umsatz']
# Remove thousand sep (.), replace decimal symbol, remove unit symbol and whitespace
df = df.replace({'\.':'', ',':'.', '\s*%\s*':'','\s*\$\s*':''}, regex=True)  
# Convert Datatype
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

In [41]:
# Rename Variables
df = df.rename(index=str, 
               columns={"Default Channel Grouping (Standard-Channelgruppierung)": "Channel", 
                        "Durchschnittl. Sitzungsdauer": "Sitzungsdauer",
                        "Hinzufügungen des Produkts zum Einkaufswagen": "Einkaufswagen",
                        "Purchase Completed (Abschlüsse für Zielvorhaben 1)": "Purchase_Completed",
                        "E-Commerce-Conversion-Rate": "Conversion_Rate",
                        "Durchschn. Preis": "Preis",
                        "Durchschn. Bestellwert": "Betsellwert"})

In [42]:
df['Alter'].value_counts() 

25-34    341
18-24    268
35-44    266
45-54    157
55-64    104
65+       84
Name: Alter, dtype: int64

In [43]:
# convert to ordinal variable
cat_Alter = CategoricalDtype(categories=
                            ['18-24', 
                             '25-34', 
                             '35-44', 
                             '45-54', 
                             '55-64',
                             '65+'],
                            ordered=True)

df.Alter = df.Alter.astype(cat_Alter)

In [44]:
# change data types (one by one for demonstrational purposes)
df.iloc[:,0] = pd.Categorical(df.iloc[:,0])
df.iloc[:,1] = pd.Categorical(df.iloc[:,1])
df.iloc[:,2] = pd.Categorical(df.iloc[:,2])
df.iloc[:,3] = pd.to_numeric(df.iloc[:,3])
df.iloc[:,4] = pd.to_numeric(df.iloc[:,4])
df.iloc[:,5] = pd.to_numeric(df.iloc[:,5])
df.iloc[:,7] = pd.to_numeric(df.iloc[:,7])
df.iloc[:,8] = pd.to_numeric(df.iloc[:,8])
df.iloc[:,9] = pd.to_numeric(df.iloc[:,9])
df.iloc[:,10] = pd.to_numeric(df.iloc[:,10])
df.iloc[:,11] = pd.to_numeric(df.iloc[:,11])
df.iloc[:,12] = pd.to_numeric(df.iloc[:,12])
df.iloc[:,13] = pd.to_numeric(df.iloc[:,13])
df.iloc[:,14] = pd.to_numeric(df.iloc[:,14])

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1220 entries, 0 to 1219
Data columns (total 15 columns):
Channel                  1220 non-null category
Alter                    1220 non-null category
Land                     1220 non-null category
Absprungrate             1220 non-null float64
Nutzer                   1220 non-null int64
Seiten/Sitzung           1220 non-null float64
Sitzungsdauer            1220 non-null object
Eindeutige Ereignisse    1220 non-null int64
Einkaufswagen            1220 non-null int64
Purchase_Completed       1220 non-null int64
Conversion_Rate          1220 non-null float64
Preis                    1220 non-null float64
Betsellwert              1220 non-null float64
Gesamtwert               1220 non-null float64
Umsatz                   1220 non-null float64
dtypes: category(3), float64(7), int64(4), object(1)
memory usage: 133.9+ KB


In [46]:
# Save clean data
# df.to_csv(PATH + 'Webanalytics_merchandise.csv')